In [1]:
from IPython.display import clear_output
!pip install mediapipe
!pip install bing-image-downloader
!pip install deepface
clear_output()

In [2]:
import os
import json
import time
import cv2
import numpy as np
import mediapipe as mp
import math
import csv 

In [3]:
#@title utils

import cv2
import numpy as np
from deepface import DeepFace
import math
import time
def deep_data_extract(img):
    embedding=None
    faces=[]
    facial_data=[]
    try:
        embedding = DeepFace.represent(img, model_name='Facenet',detector_backend='ssd')
        if embedding:
          for  i in range(len(embedding)):
            x, y, w, h = embedding[i]['facial_area']['x'], embedding[i]['facial_area']['y'], embedding[i]['facial_area']['w'], embedding[i]['facial_area']['h']
            x1, y1, x2, y2 = x, y, x+w, y+h
            faces.append((x1, y1, x2, y2 ))
            facial_data.append(embedding[i]['embedding'])
    except:
        pass
    return faces,facial_data


def rgb_to_bgr(rgb_color):
    bgr_color = (rgb_color[2], rgb_color[1], rgb_color[0])
    return bgr_color
    
def drawBox(img, x1, y1, x2, y2, l=30, t=5, rt=1, text="Unknown", id=None,display_id=False,draw_rect=False,color=(2, 240, 228),text_color=(255,255,255)):
    # Define the sci-fi style font
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 0.7
    thickness = 2
    # color = (255, 255, 255)
    color=rgb_to_bgr(color)
    text_color=rgb_to_bgr(text_color)
    # Draw the ID of the detected person on top of the bounding box
    ((id_width, id_height), _) = cv2.getTextSize(str(id), font, fontScale=fontScale, thickness=thickness)
    id_offset_x = x1 + int((x2 - x1 - id_width) / 2)
    id_offset_y = y1 - 35
    if display_id:
        cv2.putText(img, str(id), (id_offset_x, id_offset_y+25), font, fontScale, text_color, thickness)
        # Draw the name of the detected person inside the bounding box
        ((text_width, text_height), _) = cv2.getTextSize(text, font, fontScale=fontScale, thickness=thickness)
        text_offset_x = x1 + int((x2 - x1 - text_width) / 2)
        text_offset_y = y2 + 25
        cv2.putText(img, text, (text_offset_x, text_offset_y), font, fontScale, text_color, thickness)
        # Draw box around face
    if draw_rect:
        cv2.rectangle(img, (x1, y1), (x2, y2), color,thickness=rt)
    t=t-3
    face_width = x2 - x1
    face_height = y2 - y1
    # l = int(l * min(face_width, face_height) / 100)-20
    
    # Draw top-left corner
    cv2.line(img, (x1, y1), (x1 + l, y1), color, thickness=t)
    cv2.line(img, (x1, y1), (x1, y1 + l), color, thickness=t)
    # Draw top-right corner
    cv2.line(img, (x2, y1), (x2 - l, y1), color, thickness=t)
    cv2.line(img, (x2, y1), (x2, y1 + l), color, thickness=t)
    # Draw bottom-left corner
    cv2.line(img, (x1, y2), (x1 + l, y2), color, thickness=t)
    cv2.line(img, (x1, y2), (x1, y2 - l), color, thickness=t)
    # Draw bottom-right corner
    cv2.line(img, (x2, y2), (x2 - l, y2), color, thickness=t)
    cv2.line(img, (x2, y2), (x2, y2 - l), color, thickness=t)
    return img

def white_overlay(img):
    white_img = np.ones_like(img) * 255
    alpha = 0.5
    result = cv2.addWeighted(img, alpha, white_img, 1-alpha, 0)
    x1 = 60
    y1 = 60
    x2 = img.shape[1] - 60
    y2 = img.shape[0] - 60
    mid_x = (img.shape[1]) // 2
    roi = img[y1:y2, x1:x2]
    result[y1:y2, x1:x2] = roi
    return result



def fps_display(img,pTime):
    mid_x = (img.shape[1]) // 2
    fps = 0
    cTime = time.time()
    if cTime - pTime > 0:
        fps = 1 / (cTime - pTime)
    pTime = cTime
    # text = f'FPS: {int(fps)}'
    text=str(int(fps))
    font = cv2.FONT_HERSHEY_PLAIN
    font_scale = 3
    thickness = 3
    text_size = cv2.getTextSize(text, font, font_scale, thickness)[0]
    x = img.shape[1] - text_size[0] - 20
    color=rgb_to_bgr((240, 0, 148))
    cv2.putText(img, text, (mid_x, 45), font, font_scale,color, thickness)
    return img,pTime


   



Directory  /root /.deepface created
Directory  /root /.deepface/weights created


In [ ]:
shutil.rmtree("content/dataset")
!mkdir /content/dataset

In [23]:
import os
import shutil
if os.path.exists("./download"):
   shutil.rmtree("./download")
if not os.path.exists("./download"):
  os.mkdir("./download")
from bing_image_downloader import downloader
def download_image(query,limit):
  try:
    downloader.download(query, limit=limit, output_dir='./download')
  except:
    pass
celebrity_list=["boy","men","girls"]
for i in celebrity_list:
  download_image(i,100)    

clear_output()  

In [24]:
import string
import random
def generate_name():
  # Define the length of the file name
  length = 8
  # Define the set of characters to choose from
  chars = string.ascii_letters + string.digits
  # Generate a random file name
  file_name = ''.join(random.choice(chars) for _ in range(length))
  # Print the file name
  return  file_name

  
if os.path.exists("./dataset"):
    shutil.rmtree("./dataset")
#     os.mkdir("./output")
if not os.path.exists("./dataset"):
    os.makedirs("./dataset")

for i in os.listdir("./download"):
  for j in os.listdir(f"./download/{i}"):
    try:
      extenstion=j.split(".")[-1]
      shutil.copy(f"./download/{i}/{j}",f"./dataset/{generate_name()}.{extenstion}")
    except:
      pass    

In [25]:
len(os.listdir("./dataset"))

269

In [ ]:
def utils(img):
  # cv2_imshow(img)
  faces,facial_data=deep_data_extract(img)
  if len(faces)!=0 and len(facial_data)!=0:
    if len(faces)==len(facial_data):
        return facial_data[0]
  else:
    return None
        


In [ ]:
import csv
def unknown_csv(face_data):
    with open('unknown.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([face_data, "unknown",0,"None","None"])

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
for i in os.listdir("./dataset"):
  img=cv2.imread(f"./dataset/{i}")
  try:
    face_data=utils(img)
    if face_data!=None:
      print(face_data)
      unknown_csv(face_data)
  except:
    pass
clear_output()
from google.colab import files
files.download('/content/unknown.csv')

In [8]:
!pip install -q mediapipe==0.10.0

In [9]:
import cv2 
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
ImageEmbedder = mp.tasks.vision.ImageEmbedder
ImageEmbedderOptions = mp.tasks.vision.ImageEmbedderOptions
VisionRunningMode = mp.tasks.vision.RunningMode

In [10]:

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized


In [13]:
!wget https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite

--2023-06-01 17:18:29--  https://storage.googleapis.com/mediapipe-models/face_detector/blaze_face_short_range/float16/1/blaze_face_short_range.tflite
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.31.128, 142.251.111.128, 142.251.163.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 229746 (224K) [application/octet-stream]
Saving to: ‘blaze_face_short_range.tflite’

blaze_face_short_ra 100%[===================>] 224.36K  --.-KB/s    in 0.002s  

2023-06-01 17:18:29 (98.0 MB/s) - ‘blaze_face_short_range.tflite’ saved [229746/229746]



In [14]:
!wget https://storage.googleapis.com/mediapipe-models/image_embedder/mobilenet_v3_large/float32/latest/mobilenet_v3_large.tflite

--2023-06-01 17:19:27--  https://storage.googleapis.com/mediapipe-models/image_embedder/mobilenet_v3_large/float32/latest/mobilenet_v3_large.tflite
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.111.128, 142.251.163.128, 142.251.167.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10889458 (10M) [application/octet-stream]
Saving to: ‘mobilenet_v3_large.tflite’

mobilenet_v3_large. 100%[===================>]  10.38M  26.6MB/s    in 0.4s    

2023-06-01 17:19:27 (26.6 MB/s) - ‘mobilenet_v3_large.tflite’ saved [10889458/10889458]



In [15]:

BaseOptions = mp.tasks.BaseOptions
ImageEmbedder = mp.tasks.vision.ImageEmbedder
ImageEmbedderOptions = mp.tasks.vision.ImageEmbedderOptions
VisionRunningMode = mp.tasks.vision.RunningMode

new_options = ImageEmbedderOptions(
    base_options=BaseOptions(model_asset_path='mobilenet_v3_large.tflite'),
    l2_normalize=True,
    quantize=True,
    running_mode=VisionRunningMode.IMAGE)

embedder=ImageEmbedder.create_from_options(new_options)
def get_embadding(img):
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)
    embedding_result = embedder.embed(mp_image)
    embedding_list = embedding_result.embeddings[0].embedding.tolist()
    return embedding_list

import csv
import os
def save_face_data_to_csv(face_data, ids, filename):
    if not isinstance(face_data, list):
        print("Error: face_data must be a list.")
        return
    file_exists = os.path.exists(filename)
    with open(filename, 'a' if file_exists else 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if not file_exists:
            writer.writerow(["face_data", "id"])
        
        writer.writerow([face_data, ids])
    if file_exists:
        pass 
        # print(f"Face data appended to {filename} successfully.")
    else:
        pass
        # print(f"Face data saved to {filename} successfully.")
    

In [16]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [17]:
base_options = python.BaseOptions(model_asset_path='blaze_face_short_range.tflite')
options = vision.FaceDetectorOptions(base_options=base_options)
detector = vision.FaceDetector.create_from_options(options)

In [21]:
def extract_facial_data(IMAGE_FILE):
    img = cv2.imread(IMAGE_FILE)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)
    img = cv2.imread(IMAGE_FILE)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    detection_result = detector.detect(image)
    if detection_result.detections:
        for detection in detection_result.detections:
            bbox = detection.bounding_box
            x1, y1 = int(bbox.origin_x),int( bbox.origin_y)
            x2, y2 = int(bbox.origin_x + bbox.width), int(bbox.origin_y + bbox.height)
            x1, y1 = x1-20, y1-170
            x2, y2 = x2+30, y2+130
            # cv2.rectangle (img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            crop_img = img[y1:y2, x1:x2]
            crop_img = image_resize(crop_img, width=480, height=480)
            face_data=get_embadding(crop_img) 
            save_face_data_to_csv(face_data, 0, "unknown.csv")

In [ ]:
os.listdir("./dataset"):

In [26]:
import cv2
from google.colab.patches import cv2_imshow
for i in os.listdir("./dataset"):
  try:
        extract_facial_data(f"./dataset/{i}")
  except:
    pass
  
   